<a href="https://colab.research.google.com/github/youngmoo/ECES-434/blob/main/Class%205.1%20(2021-02-08).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-434: Class 5.1 (2021-02-08)**
Week 5: Halfway there!


In [2]:
import numpy as np                      # NumPy, abbreviated as np
import matplotlib.pyplot as plt         # MatplotLib PyPlot module, abbreviated as plt
from matplotlib import animation, rc    # MatplotLib animation module
%matplotlib inline
from scipy import signal                # SciPy's signal module, for DSP functions
import soundfile as sf                  # Switching to the soundfile module for reading and writing soundfiles

import IPython.display as ipd           # Interactive Python display module, for playing sounds
from IPython.display import HTML        # For displaying animations
rc('animation', html='jshtml')          # Provides animation controls

ClassPath = '/content/drive/My Drive/ECES-434 Sessions/Class 5-1/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'anonymous'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Custom plotting functions
Because we're always plotting...

## plotSpectrogram

In [4]:
def plotSpectrogram(sig, fs, win='hann', nseg=512, olap=256, fft_len=512):
  f1, t1, Sxx = signal.spectrogram(sig, fs, window=win, nperseg=nseg, noverlap=olap, nfft=fft_len)

  fig = plt.figure(figsize=(16,6))

  plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Sxx)))
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')
  return fig, plt

## myPlot(): properly formats time domain plot of a signal

In [5]:
def myPlot(sig, fs=44100):
  fig = plt.figure(figsize=(16,4))
  t = np.arange(len(sig)) / fs
  plt.plot(t, sig)
  plt.xlabel('Time (sec)')
  return fig, plt

## myPlotFFT(): properly formats frequency domain plot of a signal

In [6]:
def myPlotFFT(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,4))
  plt.plot(f, 20*np.log10(np.abs(S)))
  plt.xlim(0, x_lim)
  return fig, plt  

## myPlotFFTPhase

In [7]:
def myPlotFFTPhase(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,4))
  plt.plot(f, np.unwrap(np.angle(S)))
  plt.xlim(0, x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Phase (radians)')
  return fig, plt

## Custom FFT animation functions

In [8]:
n_o = 0
f_size = 2048
n_hop = f_size / 2
N_fft = 4096
fs = 44100
f = np.arange(N_fft) * fs / N_fft

# First set up the figure, the axis, and the plot element we want to animate
def setupAnimFFT(x_lim=(0,20000), y_lim=(-120,100)):
  fig = plt.figure(figsize=(14,6))
  ax = plt.axes(xlim=x_lim,ylim=y_lim)
  plt.close()   # Don't output the final figure separately
  line, = ax.plot([], [])
  return fig, line

# initialization function: plot the background of each frame
def initAnimFFT():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially  
def animateFFT(i, sig):
    n1 = int(n_o + n_hop*i)
    n2 = int(n_o + n_hop*i + f_size)

    x_i = sig[n1:n2]
    X_i = np.fft.fft(x_i * np.hanning(len(x_i)), n=N_fft)
    X_mag = 20*np.log(np.abs(X_i))

    line.set_data(f, X_mag)
    return (line,)  

# Usage:
# fig, line = setupAnimFFT()
# anim = animation.FuncAnimation(fig, animateFFT, init_func=initAnimFFT, frames=120, fargs=(signal,), interval=1000/30, blit=True)
# anim

# Load today's sound files

In [ ]:
s11, fs11 = sf.read(ClassPath + 'sounds/Sinatra-11kHz-mono.wav')
ipd.Audio(s11,rate=fs11)

# Resampling, again... (sorry)
Let's briefly return to our 7/5 upsampling/downsampling ratio (11025 Hz >> 15435 Hz)

In [ ]:
s77 = ...


## FIR filter based on a Hann window

In [3]:
h_fir = np.hanning(28)
h_fir = h_fir / np.sum(h_fir)

Plot the frequency response

In [ ]:
# What's the frequency respose?
myPlotFFT(...

Downsample audio and plot

In [ ]:
s15 = ...
myPlot(s15[:200], ...
ipd.Audio(s15, ...

# Signal to Noise Ratio (SNR)

In [ ]:
s15_target, fs15 = sf.read(ClassPath + 'sounds/Sinatra-15435Hz.wav')
len(s15_target)

Plot the beginning (100 samples) of the target signal vs. your resampled signal

In [ ]:
t = len(s15) / fs15
plt.plot(t[:100],s15_target[:100], ...

Compute the signal to noise ratio between the target signal and your resampled signal

In [ ]:
sig_pow = ...
sq_err = ...
snr = ...
snr_dB = ...

Plot a sample-by-sample SNR

In [ ]:
# Plot sample-by-sample SNR in dB
plt.plot()

# What about an IIR filter?


In [ ]:
b,a = signal.butter(11, 1/7)       # IIR filter design using the Butterworth prototype
h_iir = ...


Plot the phase of the IIR filter

In [ ]:
myPlotFFTPhase(h_iir, x_lim=35000, fs=fs11*7)

Filter, downsample, and plot the resampled signal

In [ ]:
s77_iir = signal.lfilter(b, a, s77)
s15_iir = ...
myPlot( ...

Plot first 100 samples of target signal and the IIR resampled signal.

In [ ]:
plt.plot(t[:100],s15_target[:100], ...
#plt.ylim(-0.005,0.005)
#plt.xlim(0.15,0.18)

Compute the SNR between target signal and the IIR resampled signal

In [ ]:
sq_err_iir = ...
snr_iir = ...
snr_dB_iir = ...

Plot the sample-by-sample SNR between the target signal and the IIR resampled signal

# What is the optimal or ideal FIR low-pass filter?



In [ ]:
fs = fs11*7
N = 1024
L = 32
H_ideal = n...

fig = plt.figure(figsize=(16,4))
f = np.arange(N) * fs / N
plt.plot(f, H_ideal)

## Is there an *inverse* Fourier Transform?
We haven't learned this yet, but let's try it...

In [ ]:
h_ideal = np.fft.ifft(H_ideal)
myPlot(np.real(h_ideal), fs=fs11*7)

In [ ]:
# Plot the imaginary part of the inverse FT

In [ ]:
# Filter needs to be symmetric
H_ideal_sym = ...

fig = plt.figure(figsize=(16,4))
f = np.arange(len(H_ideal_sym)) * fs / len(H_ideal_sym)
plt.plot(f, H_ideal_sym)

In [ ]:
h_ideal_sym = np.fft.ifft(H_ideal_sym)
myPlot(np.real(h_ideal_sym))

In [ ]:
h_ideal_shift = np.fft.fftshift(h_ideal_sym)
myPlot(np.real(h_ideal_shift))